61人训练：采用集成学习

1，61人随机分成4 fold,分别训练，预测值取平均。average MAE:  19.133

2,61人随机分成5 fold,分别训练，预测值取平均。




In [1]:
import math
import torch
import gpytorch
import numpy as np
import pandas as pd
import gc
from sklearn.metrics import mean_absolute_error
from LBFGS import FullBatchLBFGS
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupKFold


torch.set_default_tensor_type(torch.DoubleTensor)

data=pd.read_csv('QingData_sbp.csv')
#data = pd.read_csv('data_20p_1h_21f.csv')
#data=data.sample(frac=0.5,random_state=1)
patient_list=data['patient_id'].unique()
#print(patient_list)
####################################SBP 加‘20’
feature=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20']
#feature=['HrRR','HrSS','AmFS','AmSN','AmFN','AmFN_FS','AmFN_SN','TmFN','TmNF','TmFN_NF','Tm_FS','Tm_SF','Tm_SN','Tm_FQ','PAT_S','PAT_F','PAT_Q','ArFS','ArSN','ArNF','ArNF_FN']
n_devices = 3#torch.cuda.device_count()
print('Planning to run on {} GPUs.'.format(n_devices))

Planning to run on 3 GPUs.


In [2]:
MAE=0
ME=0
STD=0
L=len(patient_list)
K=4

for id in range(L):
    
    scaler=StandardScaler()
    output_device = torch.device('cuda:0')
    
    source_data=data[data['patient_id']!=patient_list[id]]
    target_data=data[data['patient_id']==patient_list[id]]
    source_x=source_data[feature]
    source_x=scaler.fit_transform(source_x)
    source_y=source_data['SBP']
    
    test_x=target_data[feature]
    test_x=scaler.transform(test_x)
    test_y=target_data['SBP']
    test_x=torch.from_numpy( test_x )
    test_y=torch.from_numpy( np.array(test_y))
    test_x, test_y = test_x.to(output_device), test_y.to(output_device)
    
    def predict(train_x, train_y):
        train_x=torch.from_numpy( train_x )
        train_y=torch.from_numpy( np.array(train_y) )
        train_x, train_y = train_x.to(output_device), train_y.to(output_device)
        checkpoint_size = find_best_gpu_setting(train_x, train_y,
                                        n_devices=n_devices,
                                        output_device=output_device,
                                        preconditioner_size=100)
        
        model, likelihood = train(train_x, train_y,
                                  n_devices=n_devices, output_device=output_device,
                                  checkpoint_size=checkpoint_size,
                                  preconditioner_size=100,
                                  n_training_iter=100)
        
        model.eval()
        likelihood.eval()
        with torch.no_grad(),\
             gpytorch.settings.fast_pred_var(),\
             gpytorch.beta_features.checkpoint_kernel(checkpoint_size):        
            y_preds = likelihood(model(test_x))
            y_mean=y_preds.mean
        
        return y_mean.cpu().numpy()
        
    

    class ExactGPModel(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood, n_devices):
            super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
            self.mean_module = gpytorch.means.ConstantMean()
            base_covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=0.5))

            self.covar_module = gpytorch.kernels.MultiDeviceKernel(
                base_covar_module, device_ids=range(n_devices),
                output_device=output_device
            )

        def forward(self, x):
            mean_x = self.mean_module(x)
            covar_x = self.covar_module(x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

    def train(train_x,
              train_y,
              n_devices,
              output_device,
              checkpoint_size,
              preconditioner_size,
              n_training_iter,
    ):
        likelihood = gpytorch.likelihoods.GaussianLikelihood().to(output_device)
        model = ExactGPModel(train_x, train_y, likelihood, n_devices).to(output_device)
        model.train()
        likelihood.train()

        optimizer = FullBatchLBFGS(model.parameters(), lr=0.1)
        # "Loss" for GPs - the marginal log likelihood
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)


        with gpytorch.beta_features.checkpoint_kernel(checkpoint_size), \
             gpytorch.settings.max_preconditioner_size(preconditioner_size):

            def closure():
                optimizer.zero_grad()
                output = model(train_x)
                loss = -mll(output, train_y)
                return loss

            loss = closure()
            loss.backward()

            for i in range(n_training_iter):
                options = {'closure': closure, 'current_loss': loss, 'max_ls': 10}
                loss, _, _, _, _, _, _, fail = optimizer.step(options)

#                 print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
#                     i + 1, n_training_iter, loss.item(),
#                     model.covar_module.module.base_kernel.lengthscale.item(),
#                     model.likelihood.noise.item()
#                 ))

                if fail:
                    print('Convergence reached!')
                    break

        print(f"Finished training on {train_x.size(0)} data points using {n_devices} GPUs.")
        return model, likelihood
    
    def find_best_gpu_setting(train_x,
                              train_y,
                              n_devices,
                              output_device,
                              preconditioner_size
    ):
        N = train_x.size(0)

        # Find the optimum partition/checkpoint size by decreasing in powers of 2
        # Start with no partitioning (size = 0)
        settings = [0] + [int(n) for n in np.ceil(N / 2**np.arange(1, np.floor(np.log2(N))))]

        for checkpoint_size in settings:
            print('Number of devices: {} -- Kernel partition size: {}'.format(n_devices, checkpoint_size))
            try:
                # Try a full forward and backward pass with this setting to check memory usage
                _, _ = train(train_x, train_y,
                             n_devices=n_devices, output_device=output_device,
                             checkpoint_size=checkpoint_size,
                             preconditioner_size=preconditioner_size, n_training_iter=1)

                # when successful, break out of for-loop and jump to finally block
                break
            except RuntimeError as e:
                print('RuntimeError: {}'.format(e))
            except AttributeError as e:
                print('AttributeError: {}'.format(e))
            finally:
                # handle CUDA OOM error
                gc.collect()
                torch.cuda.empty_cache()
        return checkpoint_size

    prediction=np.zeros(target_data.shape[0])
    groups=np.array(source_data['patient_id'])
    gkf=GroupKFold(n_splits=K)
    for _ , train_index in gkf.split(source_x, source_y, groups):
        print("TRAIN:", np.unique(groups[train_index]))
        train_x, train_y = source_x[train_index], source_y.iloc[train_index]
        prediction+=predict(train_x,train_y)
    prediction=prediction/K
    mae=mean_absolute_error(test_y.cpu(),prediction)
    MAE+=mae  
    error=(test_y.cpu().numpy()-prediction)
    me=np.mean(error)
    ME+=me
    std=np.std(error,ddof=1)
    STD+=std*(test_x.cpu().shape[0]-1)          
    print(patient_list[id],'MAE: ', format(mae,'.3f'),'ME',format(me,'.3f'),'STD: ',format(std,'.3f'))



print('average MAE: ', format(MAE/L,'.3f'),'ME: ', format(ME/L,'.3f'), 'STD: ', format(STD/(data.shape[0]-L),'.3f'))

TRAIN: [ 0  3  5 16 24 28 31 42 43 44 51 53 55 57 59]
Number of devices: 3 -- Kernel partition size: 0


/home/staff_qing_liu/Jiacheng /LBFGS.py:257: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /opt/conda/conda-bld/pytorch_1616554788289/work/torch/csrc/utils/python_arg_parser.cpp:1005.)
  p.data.add_(step_size, update[offset:offset + numel].view_as(p.data))


Finished training on 2482 data points using 3 GPUs.
Convergence reached!
Finished training on 2482 data points using 3 GPUs.
TRAIN: [ 8 14 21 25 26 27 29 32 33 35 37 49 50 52 61]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2496 data points using 3 GPUs.
Convergence reached!
Finished training on 2496 data points using 3 GPUs.
TRAIN: [10 11 12 13 17 20 22 30 38 39 40 46 54 56 60]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2482 data points using 3 GPUs.
Convergence reached!
Finished training on 2482 data points using 3 GPUs.
TRAIN: [ 2  4  6  7  9 15 18 19 23 34 36 41 45 47 48 58]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2496 data points using 3 GPUs.
Convergence reached!
Finished training on 2496 data points using 3 GPUs.
1 MAE:  25.814 ME -25.633 STD:  12.464
TRAIN: [ 4  6 10 11 14 17 25 29 32 35 40 41 44 52 61]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2478 data points using 3 

TRAIN: [ 2  4  7 24 25 29 32 35 36 37 45 47 52 57 58]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2498 data points using 3 GPUs.
Convergence reached!
Finished training on 2498 data points using 3 GPUs.
9 MAE:  29.102 ME 24.714 STD:  21.815
TRAIN: [12 13 20 28 30 31 33 39 41 42 44 50 53 56 60]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2481 data points using 3 GPUs.
Convergence reached!
Finished training on 2481 data points using 3 GPUs.
TRAIN: [ 3  8 14 15 18 19 21 23 24 26 27 34 36 48 57 59]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2495 data points using 3 GPUs.
Convergence reached!
Finished training on 2495 data points using 3 GPUs.
TRAIN: [ 1  6  9 11 17 22 25 29 32 35 38 46 51 52 54]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2494 data points using 3 GPUs.
Convergence reached!
Finished training on 2494 data points using 3 GPUs.
TRAIN: [ 0  2  4  5  7 16 37 40 43 45 47 49 55 

Finished training on 2502 data points using 3 GPUs.
Convergence reached!
Finished training on 2502 data points using 3 GPUs.
TRAIN: [ 3 10 11 12 13 17 22 29 30 38 39 40 46 56 61]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2488 data points using 3 GPUs.
Convergence reached!
Finished training on 2488 data points using 3 GPUs.
TRAIN: [ 0  2  4  6  7  9 25 32 33 37 45 47 48 49 58]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2487 data points using 3 GPUs.
Convergence reached!
Finished training on 2487 data points using 3 GPUs.
18 MAE:  41.496 ME 41.496 STD:  16.991
TRAIN: [15 23 28 29 31 34 35 41 42 44 50 51 53 54 56 57]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2508 data points using 3 GPUs.
Convergence reached!
Finished training on 2508 data points using 3 GPUs.
TRAIN: [ 0  1  8 13 14 16 18 20 21 24 26 27 36 59 60]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2495 data points using 3 

26 MAE:  13.778 ME -13.059 STD:  11.523
TRAIN: [12 18 25 31 32 36 38 41 42 43 44 49 51 52 53]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2491 data points using 3 GPUs.
Convergence reached!
Finished training on 2491 data points using 3 GPUs.
TRAIN: [ 3  6  8 11 15 16 21 23 24 26 28 34 35 40 48 59]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2491 data points using 3 GPUs.
Convergence reached!
Finished training on 2491 data points using 3 GPUs.
TRAIN: [ 0  1  5  7 17 19 20 22 37 45 46 50 55 57 61]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2476 data points using 3 GPUs.
Convergence reached!
Finished training on 2476 data points using 3 GPUs.
TRAIN: [ 2  4  9 10 13 14 29 30 33 39 47 54 56 58 60]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2477 data points using 3 GPUs.
Convergence reached!
Finished training on 2477 data points using 3 GPUs.
27 MAE:  17.066 ME -14.005 STD:  14.617
TRAIN

Finished training on 2488 data points using 3 GPUs.
Convergence reached!
Finished training on 2488 data points using 3 GPUs.
TRAIN: [ 0  2  4  6  7  9 25 32 33 37 45 47 49 56 58]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2487 data points using 3 GPUs.
Convergence reached!
Finished training on 2487 data points using 3 GPUs.
35 MAE:  19.690 ME -19.392 STD:  13.854
TRAIN: [ 3 12 13 27 29 30 31 39 42 44 48 51 53 59 60]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2482 data points using 3 GPUs.
Convergence reached!
Finished training on 2482 data points using 3 GPUs.
TRAIN: [ 6  8  9 10 14 15 18 19 21 23 26 28 33 34 41 56]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2496 data points using 3 GPUs.
Convergence reached!
Finished training on 2496 data points using 3 GPUs.
TRAIN: [ 0  1  5 11 16 17 20 22 24 38 46 50 54 55 57]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2481 data points using 3

TRAIN: [ 1  6  9 10 11 21 25 32 35 42 43 45 46 52 54]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2471 data points using 3 GPUs.
Convergence reached!
Finished training on 2471 data points using 3 GPUs.
TRAIN: [ 0  2  3  5  7  8 16 20 38 40 49 51 55 58 61]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2457 data points using 3 GPUs.
Convergence reached!
Finished training on 2457 data points using 3 GPUs.
TRAIN: [14 15 18 19 22 23 24 28 31 33 34 41 47 50 56 57]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2471 data points using 3 GPUs.
Convergence reached!
Finished training on 2471 data points using 3 GPUs.
44 MAE:  14.722 ME -0.996 STD:  18.348
TRAIN: [ 0  3  5 11 16 20 24 27 31 33 42 44 55 57 59]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2479 data points using 3 GPUs.
Convergence reached!
Finished training on 2479 data points using 3 GPUs.
TRAIN: [ 8 12 13 21 26 28 29 30 39 40 41 48 50

Finished training on 2525 data points using 3 GPUs.
Convergence reached!
Finished training on 2525 data points using 3 GPUs.
52 MAE:  40.824 ME 31.396 STD:  32.593
TRAIN: [11 12 13 27 29 30 31 33 39 41 42 44 48 59 60]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2480 data points using 3 GPUs.
Convergence reached!
Finished training on 2480 data points using 3 GPUs.
TRAIN: [ 0  3  5  8 16 20 21 24 26 28 40 45 50 55 57]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2479 data points using 3 GPUs.
Convergence reached!
Finished training on 2479 data points using 3 GPUs.
TRAIN: [ 1  6  7  9 17 22 25 32 35 38 43 46 51 52 54]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2493 data points using 3 GPUs.
Convergence reached!
Finished training on 2493 data points using 3 GPUs.
TRAIN: [ 2  4 10 14 15 18 19 23 34 36 37 47 49 56 58 61]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2493 data points using 3 

TRAIN: [ 3 11 12 13 17 22 29 30 38 39 40 46 48 50 60]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2486 data points using 3 GPUs.
Convergence reached!
Finished training on 2486 data points using 3 GPUs.
TRAIN: [ 2  4  7  9 14 25 32 33 35 37 43 47 52 57 58]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2499 data points using 3 GPUs.
Convergence reached!
Finished training on 2499 data points using 3 GPUs.
61 MAE:  18.313 ME -17.846 STD:  13.204
TRAIN: [13 20 28 31 35 41 42 44 48 50 51 52 53 54 57]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2515 data points using 3 GPUs.
Convergence reached!
Finished training on 2515 data points using 3 GPUs.
TRAIN: [ 1  8 10 16 18 21 24 25 26 27 33 39 43 45 60]
Number of devices: 3 -- Kernel partition size: 0
Finished training on 2500 data points using 3 GPUs.
Convergence reached!
Finished training on 2500 data points using 3 GPUs.
TRAIN: [ 3  5 11 12 15 17 22 30 32 34 38 40 46 5